In [1]:
import numpy as np
import pandas as pd

pd.options.plotting.backend = 'plotly'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

import numpy as np

from scipy.optimize import minimize
from scipy.stats import poisson

In [2]:
def price_clean(x: str) -> float:
    
    """
    Esta función limpia una string, quitando $ y la coma y devuelve un float
    
    param x: string a limpiar
    
    return: float
    """
    x=str(x).replace('$', '').replace(',', '')
    return float(x)

In [3]:
def loglikelihood(x: int, price: float, theta_1: float, theta_0: float) -> float:
    """
    The log-likelihood is the model function to be maximize for the parameter extraction (MLE).
    
    param x: airbnb count (by day), int
    param price: average price (by day), float
    param theta_1: parameter of demand function (exp(theta_1 * price + theta_0)), float
    param theta_0: intercept, float
    
    return: log-likelihood evaluation, float
    """
    lamb = np.exp(theta_1 * price + theta_0)

    return -lamb + x * np.log(lamb)

In [4]:
def fit(data: pd.DataFrame, method: str='trust-constr') -> dict:
    """
    Fit function to obtain demand parameters.
    
    data: pd.DataFrame(), needs columns 'count_' and 'price'
    method: optimization method, str (for scipy minimize)
    
    return: dictionary with parameters
    """
    
    # initial values for theta_1 and theta_0
    val_initial = np.array([0, 0])
    
    # minus loglikelihood, function to be minimize
    def minus_loglike(params):
        
        theta_1 = params[0]
        theta_0 = params[1]

        log_like = [loglikelihood(n.count_, n.avg_price, theta_1, theta_0) for n in data.itertuples()]

        return -sum(log_like)

    output = minimize(minus_loglike, val_initial, method=method)

    return dict(zip(['theta_1', 'theta_0'], output.x))

In [5]:
def extract_data(available='t', lower=0, upper=10000) -> pd.DataFrame:
    """
    
    """
    listings=pd.read_csv('../data/raw_data/listings.csv.gz', compression='gzip', low_memory=False,
                         usecols=['id', 'neighbourhood_group_cleansed'])
    
    calendar=pd.read_csv('../data/raw_data/calendar.csv.gz', compression='gzip', low_memory=False,
                         usecols=['listing_id', 'available', 'date', 'price'])
    
    
    data=calendar.merge(listings, left_on='listing_id', right_on='id')

    data.price=data.price.apply(price_clean)

    data=data[(data.price>lower) & (data.price<upper)]

    data=data[data.available==available]
    
    df=data.groupby(['date', 'neighbourhood_group_cleansed']).agg({'date': 'count', 'price': 'mean'})
    
    df.columns=['count_', 'avg_price']
    
    df.reset_index(inplace=True)
    
    df.columns=['date', 'neighbourhood', 'count_', 'avg_price']
    
    df.date=pd.to_datetime(df.date)
    
    return df

In [6]:
def neighbourhood(df: pd.DataFrame, b: str) -> dict:
    print(b)
    df_barrio=df[df.neighbourhood==b]
    
    df_barrio.avg_price=df_barrio.avg_price/100
    
    all_dates=pd.DataFrame(pd.date_range(df.date.min(), df.date.max()), columns=['date'])
    all_dates[['count_', 'price_avg']] = 0, 0.9
    
    df_barrio=all_dates.merge(right=df_barrio, how='left', on='date')

    df_barrio.count__y.fillna(df_barrio.count__x, inplace=True)
    df_barrio.avg_price.fillna(df_barrio.price_avg, inplace=True)
    df_barrio.neighbourhood.fillna(b, inplace=True)

    df_barrio=df_barrio[['date', 'count__y', 'avg_price']]

    df_barrio.rename(columns={'count__y': 'count_'}, inplace=True)
    
    
    fig=df_barrio.plot.scatter(x='date', y='count_', title=b)
    fig.show();
    
    params=fit(df_barrio)
    
    prices=[round(0.1 + i*0.01, 2) for i in range(500)]

    demands=[np.exp(params['theta_1'] * p + params['theta_0']) for p in prices]

    results=pd.DataFrame()

    results['price']=[p*100 for p in prices]

    results['demand']=demands

    results['income']=results.price * results.demand
    
    optimus=0
    if params['theta_1']<0:
        optimus=results[results.income==results.income.max()].price.values[0]
    else:
        optimus=df_barrio.avg_price.mean()*100
    
    fig=results.plot(kind='scatter', x='price', y='demand', title=b)
    fig.show();
    
    fig=results.plot(kind='scatter', x='price', y='income', title=b)
    fig.show();
    
    display(results[results.income==results.income.max()])
    
    
    return  {'barrio': b, 
             'optimo': optimus,
             'params': params, 
             'converge': params['theta_1']<0,
             'describe': df_barrio.describe().to_dict(),
             'average': df_barrio.avg_price.mean()}


**Arganzuela**

In [20]:
%%time

data=extract_data(available='t', lower=50, upper=10000)

CPU times: user 5.61 s, sys: 762 ms, total: 6.38 s
Wall time: 6.74 s


In [14]:
barrios=data.neighbourhood.unique()

In [15]:
barrios

array(['Arganzuela', 'Carabanchel', 'Centro', 'Chamartín', 'Chamberí',
       'Ciudad Lineal', 'Moncloa - Aravaca', 'Puente de Vallecas',
       'Retiro', 'Salamanca', 'San Blas - Canillejas', 'Tetuán', 'Usera',
       'Villaverde', 'Barajas', 'Fuencarral - El Pardo', 'Hortaleza',
       'Latina', 'Moratalaz', 'Vicálvaro', 'Villa de Vallecas'],
      dtype=object)

In [21]:
neighbourhood(data, 'Arganzuela')

Arganzuela


,price,demand,income
499,509.0,184.306115,93811.812544


{'barrio': 'Arganzuela',
 'optimo': 509.0,
 'params': {'theta_1': -0.1046482787966317, 'theta_0': 5.749257782889966},
 'converge': True,
 'describe': {'count_': {'count': 366.0,
   'mean': 271.30054644808746,
   'std': 56.42167415780101,
   'min': 4.0,
   '25%': 225.25,
   '50%': 258.0,
   '75%': 315.0,
   'max': 397.0},
  'avg_price': {'count': 366.0,
   'mean': 1.3961213462685407,
   'std': 0.11408796658772825,
   'min': 0.9175,
   '25%': 1.3308102243390012,
   '50%': 1.3758810120811487,
   '75%': 1.4491837063102109,
   'max': 1.834047619047619}},
 'average': 1.3961213462685407}

**Salamanca**

In [17]:
%%time

# Salamanca

data=extract_data(available='t', lower=24, upper=10000)

neighbourhood(data, 'Salamanca')

Salamanca


,price,demand,income
347,357.0,356.482224,127264.153854


CPU times: user 5.99 s, sys: 686 ms, total: 6.68 s
Wall time: 6.68 s


{'barrio': 'Salamanca',
 'optimo': 357.0,
 'params': {'theta_1': -0.27994499390113986, 'theta_0': 6.8756880033860215},
 'converge': True,
 'describe': {'count_': {'count': 366.0,
   'mean': 601.5300546448087,
   'std': 76.65888093202528,
   'min': 18.0,
   '25%': 557.0,
   '50%': 584.0,
   '75%': 655.75,
   'max': 761.0},
  'avg_price': {'count': 366.0,
   'mean': 1.7036343262801534,
   'std': 0.13609348083158262,
   'min': 1.3516666666666666,
   '25%': 1.6165646320886216,
   '50%': 1.6719499105545617,
   '75%': 1.7694378462313245,
   'max': 2.3476344086021506}},
 'average': 1.7036343262801534}

**Latina**

In [31]:
%%time

# Latina

data=extract_data(available='t', lower=24, upper=10000)

neighbourhood(data, 'Latina')

Latina


,price,demand,income
499,509.0,631.90555,321639.924987


CPU times: user 5.99 s, sys: 820 ms, total: 6.81 s
Wall time: 7.21 s


{'barrio': 'Latina',
 'optimo': 137.64500846612515,
 'params': {'theta_1': 0.307665137227735, 'theta_0': 4.882724388402085},
 'converge': False,
 'describe': {'count_': {'count': 366.0,
   'mean': 202.7568306010929,
   'std': 36.794928102076504,
   'min': 0.0,
   '25%': 167.0,
   '50%': 195.5,
   '75%': 231.0,
   'max': 272.0},
  'avg_price': {'count': 366.0,
   'mean': 1.3764500846612515,
   'std': 0.34621281385708524,
   'min': 0.76,
   '25%': 1.0020884146341462,
   '50%': 1.4031872124340703,
   '75%': 1.5786491200608241,
   'max': 2.3402857142857143}},
 'average': 1.3764500846612515}